#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-11-29 14:27:55--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc62012bb6b368a25c7f4f94960d.dl.dropboxusercontent.com/cd/0/inline/CfQ-G7NsSy-RcsDUyOz3HxaZC4w960_Elk--wzfCutExzNdj1ZDfPmZzo8QpeEifGwngT7-lJmRnvGeDzmFU47WVopPAdan8TbImj7pr6QpfJKbFT8RkPj3kZilehOd4LyM/file?dl=1# [following]
--2024-11-29 14:27:56--  https://uc62012bb6b368a25c7f4f94960d.dl.dropboxusercontent.com/cd/0/inline/CfQ-G7NsSy-RcsDUyOz3HxaZC4w960_Elk--wzfCutExzNdj1ZDfPmZzo8QpeEifGwngT7-lJmRnvGeDzmFU47WVopPAdan8TbImj7pr6QpfJKbFT8RkPj3kZilehOd4LyM/file?dl=1
Resolving uc62012bb6b368a25c7f4f94960d.dl.dropboxusercontent.com (uc62012bb6b368a25c7f4f94960d.dl.dropboxusercontent.com)... 162.125.65.15, 2620:100:6022:15::

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()



In [3]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.0 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [7]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = f"Pretend you are a movie recommender system. I will give you a conversation between a human and assistant. Based on the conversation, you reply me with 10 recommendations without extra sentences. Here is the conversation: {conversation}."
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [8]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [9]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [10]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [11]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.01 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [12]:
prompts = []
ground_truth = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  conversation_id = user_data['Conversation'][0]['conversation_1']['conversation_id']
  prompt = generate_zeroshot(conversation_id, conversations)
  prompts.append(prompt)


In [15]:
results = text_generator(prompts, max_new_tokens=300, return_full_text=False)


In [17]:
answers = [result[0]['generated_text'] for result in results]
for answer in answers:
  print(answer)


 Based on your preference for action-adventure movies with a more aesthetically pleasing male lead, here are ten recommendations:

1. Indiana Jones series
2. Lara Croft: Tomb Raider
3. King Kong (2005)
4. The Mummy series
5. Clash of the Titans (1981)
6. Conan the Barbarian (1982)
7. Gladiator
8. 300
9. Troy
10. The Scorpion King

These movies offer a mix of action, adventure, and historical or mythological settings, with leading men who are considered attractive and charismatic. Enjoy your movie-watching experience!
 Based on your preferences and the conversation, here are 10 Liam Neeson movie recommendations:

1. Taken 2
2. Taken 3
3. Unknown
4. Non-Stop
5. A Walk Among the Tombstones
6. The Grey
7. Run All Night
8. The Commuter
9. The A-Team
10. Kinsey (a different role for Liam Neeson, not action-oriented)
 Here are 10 movie recommendations based on the user's preferences:

1. "The Great Escape" - A classic war film with a strong storyline and well-developed characters.
2. "Saving 

In [18]:
import re

def parse_recommendations(answer):
  pattern = r'\d+\.\s+"*([^"]+?)"*(?:\s\(\d{4}\))?(?:\s*-\s*.+)?(?:\s*\(\d-\d\))?$'
  recommendations = re.findall(pattern, answer, re.MULTILINE)
  return recommendations

In [19]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [27]:
def evaluate_model(answers, users):
    assert len(answers) == len(users)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 10:
         continue
       user_data = read_user_data(path_movie_data, users[i])
       user_might_like = [item_map[id] for id in user_data["user_might_like"]]
       rel_vector = np.isin(recommendations, user_might_like, assume_unique=True).astype(int)

       mean_ndcg += ndcg_at_k(rel_vector, 20, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 20, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [28]:
ndcg, recall = evaluate_model(answers, sampled_users)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

NDCG: 0.0000
Recall: 0.0000
